<a href="https://colab.research.google.com/github/ArtemChemist/CFU_counter/blob/main/YOLO_infer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.8/699.8 kB 6.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import matplotlib.pyplot as plt

import os
import glob
from datetime import datetime
import time
import cv2
import shutil

import json

from ultralytics import YOLO

In [3]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')


# Import custom module with helper functions
import sys
sys.path.append('/content/drive/My Drive')
import funcs_yolo as f

Mounted at /content/drive


In [4]:
# Unzip archives with images
!unzip -q "/content/drive/MyDrive/raw_train_val.zip" -d "/content"
!unzip -q "/content/drive/MyDrive/raw_train.zip" -d "/content/train"

In [5]:
p = os.path.abspath('.')
input_dir = os.path.join(p, 'raw_test_val')
output_dir = os.path.join(p, 'drive', 'MyDrive', 'results')
if not os.path.exists(output_dir):
  os.mkdir(output_dir)

In [6]:
model_path = os.path.join(p, 'drive', 'MyDrive', 'YOLO', 'runs', 'detect', 'train', 'weights', 'best.pt' )
model = YOLO(model_path)

In [7]:
expert_lbls = {}

ref_path = os.path.join(p, 'train', 'raw', 'images')
train_ims = []
result_txt = {}

for file in glob.glob(f'{ref_path}/*.jpg'):
    train_ims.append(file.split('/')[-1])

colors = {0: (0, 0, 0),
          1: (255, 0, 0),
          2: (0, 255, 0),
          3: (0, 0, 255),
          4: (50, 50, 50),
          5: (100, 0, 0),
          6: (0, 0, 100),
          7: (0, 100, 0),
          8: (70, 70, 70)}

cls_names = {0: 'artifact',
            1: 'bubble',
            2: 'colony',
            3: 'gate',
            4: 'lock',
            5: 'sharpie',
            6: 'star',
            7: 'tape',
            8: 'unlock'}

for file_path in glob.glob(f'{input_dir}/*.jpg'):
    filename = file_path.split('/')[-1]
    num_colonies = int(filename.split('-')[2].strip().split('.')[0])
    if filename not in train_ims:
      result = model(file_path, verbose = False)  # return a list of Results objects
      print(filename, num_colonies)
      expert_lbls[filename] = num_colonies
      pic = cv2.imread(file_path)
      result_txt[filename] = []
      for box, cls in zip(result[0].boxes.xyxy, result[0].boxes.cls):
        if int(cls) == 2:
          xB = int(box[2])
          xA = int(box[0])
          yB = int(box[3])
          yA = int(box[1])
          cv2.rectangle(pic, (xA, yA), (xB, yB), colors[int(cls)], 4)
      cv2.imwrite(os.path.join(output_dir, filename), pic)

      for cls, conf in zip(result[0].boxes.cls, result[0].boxes.conf):
        result_txt[filename].append((int(cls), float(conf)))

WARNING ⚠️ NMS time limit 0.550s exceeded
3280163292dcea778401787c12be432c-168-0.jpg 0
868daf30506b3e7686f5d7a248bfd3b0-195-0.jpg 0
937c384c38474245ae40bf034efff339-294-0.jpg 0
098bb76122bfca0ea2090035897a7732-198-0.jpg 0
79df8d99cae4c3c715beebe17a8a9770-196-0.jpg 0
7fd51e48fab0ec9e82715740fe55d3c8-80-50.jpg 50
78634fe4c187135f0aee0cc7fa4e5195-170-0.jpg 0
b9d31a39239f2590c6c70cd33c209642-192-0.jpg 0
28afcdd9970dd49da5a6a46c5090940a-190-0.jpg 0
2947aee8250f50091f1b0b5e3787302b-171-1.jpg 1
166d239028b4c7be02188c5ec46ea59d-102-0.jpg 0
6109399bdbcf388bcca22abae29fa9cb-296-0.jpg 0
a874ec35023a98aa4e8bccc275d35331-221-0.jpg 0
dbe31b6eb6b83c48f6f68b3b7cd1201d-190-0.jpg 0
a36c0324673a0b10e6ba518d41c5c347-294-0.jpg 0
a2eb3e163e8eb236a0e246ba6e27decb-99-0.jpg 0
c80fdf7412ccb7491f374c9c153a73e9-188-0.jpg 0
0a3005efc1cd464a847c0d5f503af0d8-99-0.jpg 0
49a43fefd53a39bfcf39e7e383b15b55-193-0.jpg 0
8d528f735a8c04fe8252dab6e28d1612-102-0.jpg 0
db3734340527b2dcb093637a0d44d825-214-0.jpg 0
cf289139976cbd

In [8]:
with open( os.path.join(output_dir, 'json', 'model_lbl.json'), 'w+') as f:
   json.dump(result_txt, f)

with open( os.path.join(output_dir, 'json', 'expert_lbl.json'), 'w+') as f:
   json.dump(expert_lbls, f)

with open( os.path.join(output_dir, 'json', 'legend.json'), 'w+') as f:
   json.dump(cls_names, f)

In [9]:
FP = 0
TP = 0
FN = 0
TN = 0
count = 0
P = 0

train_names = [x.split('_')[0] + '.jpg' for x in train_ims]

for key, value in expert_lbls.items():
  if key not in train_names:
    predictions = [box[0] for box in result_txt[key]]
    count +=1
    if value >0:
      P +=1
      if (2 in predictions):
        TP +=1
      else:
        FN +=1
    else:
      if (2 in predictions):
        FP +=1
      else:
        TN +=1
print(f'Total: {count}, Positives: {P}, Recall: {round(100*TP/P, 2)}%, Precision: {round(100*TP/(TP + FP), 2)}%')
print(f'FP: {FP}, TP: {TP}, FN: {FN}, TN: {TN}')

Total: 3202, Positives: 204, Recall: 99.02%, Precision: 72.66%
FP: 76, TP: 202, FN: 2, TN: 2922


In [10]:
# Save results
result_dir = os.path.join(p, "drive", "MyDrive", 'YOLO')
y_true = np.array([(x >0) for x in expert_lbls.values()])
y_pred = []
for key, value in result_txt.items():
  predictions = [box[1] for box in value if box[0]==2]
  if len(predictions) == 0:
     predictions = [0]
  y_pred.append(max(predictions))
y_pred  = np.array(y_pred)
thresholds = np.arange(0, 1, 0.005)

precisions = []
recalls = []
P = sum(y_true)
for threshold in thresholds:
  TP = sum((y_pred > threshold) & y_true)
  FP = sum((y_pred > threshold) & ~y_true)
  precisions.append(TP/(TP+FP))
  recalls.append(TP/P)

import csv
with open(os.path.join(output_dir, 'json', 'prec_rec.csv'), 'w+') as file_prrec:
  csvwriter = csv.DictWriter(file_prrec, delimiter='\t', fieldnames=["threshold", "precision", "recall"])
  csvwriter.writeheader()
  for thr, pr, rec in zip(thresholds, precisions, recalls):
    csvwriter.writerow({"threshold":thr,
                        "precision":pr,
                        "recall":rec})